# 0. 강화학습을 이용한 체스 (Reinforcement Learning Chess)

※본 커널은 캐글을 통해 공유된 https://www.kaggle.com/arjanso/reinforcement-learning-chess-1-policy-iteration 의 내용을 기반으로 작성되었습니다.

체스를 다루는 것은 거대한 상태공간(state-space)을 다루는 일이기 때문에 매우 도전적인 일(challenge)입니다. (이제는 훨씬 더 거대한 상태공간을 갖는 바둑의 세계도 점령당했지만요!) 간단한 형태의 체스로 시작하여 기본 강화학습 기술로 이러한 문제를 해결합니다. 점진적으로 체스의 실제 게임을 이해할 수 있는 체스 AI로 점차 확장할 것입니다. 여기서 말하는 간단한 형태의 체스는 다음과 같습니다.

1. 체스 말 움직이기
- 목표: 체스 판에서 두 사각형 사이의 최단 경로를 찾는 법을 배웁니다.
- 동기: 체스 말을 움직이는 것은 상태공간이 작기 때문에 간단한 강화학습 알고리즘으로 해결할 수 있습니다.
- 개념: 동적 프로그래밍(Dynamic Programming), 정책 평가(Poliy Evaluation), 정책 개선(Policy Improvement), 정책 반복(Policy Iteration), 가치 반복(Value Iteration), 동기 및 비동기 백업(Synchronous & Asynchronous back-ups), 몬테 카를로(Monte Carlo) 예측, MC 제어, 시간 차이(Temporal Difference) 학습, TD 제어, TD-람다(lambda), SARSA(-max)
2. 체스 말 잡기
- 목표: 총 이동수(fullmoves) n 내에서 상대방으로부터 많은 조각을 잡는 것
- 동기: 체스 말을 잡는 것은 win-lose-draw 이벤트보다 더 자주 발생합니다. 이를 통해 알고리즘에게 추가 정보가 제공됩니다.
- 개념: Q-러닝(Q-learning), 가치 함수 근사(value function approximation), 재실험(experience replay), 고정 q-목표(fixed-q-targets), 정책 그래디언트(policy gradients), REINFORCE, 행위자 비평(actor-critic)
3. 실제 체스
- 목표: 인간 초보자와 경쟁하여 체스를 둡니다.
- 동기: 강화학습 체스 AI
- 개념: 몬테 카를로 탐색 트리(Monte Carlo Tree Search)

이 커널에서는 강화학습 이론을 설명하고 참고하지만, 완전히 설명하지는 않습니다. 관련하여서는 다음의 유튜브와 책을 참조해주세요.
1. Reinforcement Learning: An Introduction, Richard S. Sutton and Andrew G. Barto, 1st Edition, MIT Press, march 1998
2. [RL Course by David Silver: Lecture playlist] (https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ)


# 1. 체스 말 움직이기

먼저 심플한 첫번째 문제입니다. 체스 말을 움직이는 것입니다. 체스 환경을 다룰  수 있는 python-chess라는 패키지와 RLC라는 강화학습 패키지를 활용합니다.

In [0]:
import numpy as np 
import pandas as pd 
import os
import inspect

!pip install python-chess  
!pip install --upgrade git+https://github.com/arjangroen/RLC.git  

  Cloning https://github.com/arjangroen/RLC.git to /tmp/pip-req-build-52pgzt6m
  Running command git clone -q https://github.com/arjangroen/RLC.git /tmp/pip-req-build-52pgzt6m
  Created wheel for RLC: filename=RLC-0.3-cp36-none-any.whl size=22568 sha256=8e9dfaad23ae5a756e9d544ab6ed6b13725530976610b0212d507db2c8bffec4
  Stored in directory: /tmp/pip-ephem-wheel-cache-t86c3h4v/wheels/04/68/a5/cb835cd3d76a49de696a942739c71a56bfe66d0d8ea7b4b446
Successfully built RLC


In [0]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


여기서 제가 평소에 보지 못했던 2개의 매직 셀 커멘드들이 사용되었습니다.

첫번째 

두번째 `%autoreload`의 경우, 모듈을 자동적으로 리로드(reload)하는 것인데요, pip install과 같이 ipynb 내에서 설치를 하였을 경우 새로 설치된 모듈을 제대로 불러오지 못하는 경우가 있습니다. 특히 여기서는 RLC 깃으로부터 설치하는 부분이 그러할 것으로 생각되네요. 이를 위해서 사용하는 것으로 보입니다. autoreload에 파라미터를 전달할 수 있습니다.

- `%autoreload`: 지금 바로 자동적으로 모든 모듈을 리로드합니다. 모든 모듈을 리로드합니다.(`%aimport`에서 제외된 모듈을 제외하고)
- `%autoreload 0`: 자동 리로딩 기능을 끕니다.
- `%autoreload 1`: 파이썬 코드를 실행하기 전에 매번 `%aimport` 내의 모든 모듈들을 리로드합니다.
- `%autoreload 2`: 파이썬 코드를 실행하기 전에 매번 모든 모듈을 리로드합니다.(`%aimport`에서 제외된 모듈을 제외하고)

새로운게 또 등장했네요,

`%aimport`는 자동적으로 import할 또는 하지 않을 모듈의 리스트를 나타냅니다. 여기서는 특별히 사용하지 않는 것으로 보아, 모든 모듈을 리로드하는 것으로 볼 수 있습니다.

## 1.0.1 환경 (Environment)

- 상태공간은 8x8의 그리드입니다
- 시작 상태 S는 왼쪽 위 사각형 (0, 0)을 기준으로 합니다.
- 종료 상태 F의 좌표는 (5, 7) 입니다.
- 한 상태에서 다른 상태로 이동할 때마다 보상(reward)을 1 감소시킵니다.
- 이 환경에 대한 최상의 정책은 가장 적은 수의 움직임으로 S로부터 F까지 움직이는 것입니다.

In [0]:
from RLC.move_chess.environment import Board
from RLC.move_chess.agent import Piece
from RLC.move_chess.learn import Reinforce

In [0]:
env = Board()
env.render()
env.visual_board

[['[S]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[F]', '[ ]', '[ ]']]

Board는 위에서 말한 체스판의 환경을 구성하는 클래스입니다. render()를 실행하지 않으면, env는 visual_board를 갖고 있지 않습니다. render() 함수는 시각화를 위한 초기화 함수로 보아도 좋을 것 같습니다. 즉, 내부적으로 움직일 때에는 시각화를 위한 연산까지는 필요하지 않기 때문에 기본적으로는 render()를 실행하지 않아서 연산을 최소화합니다.

## 1.0.2. 에이전트(Agent)

- 체스에서 에이전트는 체스 말입니다. (킹, 퀸, 룩, 나이트, 비숍 등)
- 에이전트는 에이전트가 어떤 상태에서 무엇을 하는지 결정하는 행동 정책을 갖고 있습니다.

In [0]:
p = Piece(piece='king')
print(p)

## 1.0.3. 강화 (Reinforce)

- 강화 객체에는 체스 말을 움직이는 문제를 해결하기 위한 알고리즘이 포함되어있습니다.
- 에이전트와 환경은 강화 객체의 속성(attributes)입니다.

In [0]:
r = Reinforce(p,env)

# 1.1. 상태 평가(State Evaluation)

## 1.1.1. 이론

에이전트가 보상을 최적화하도록 하려면, 보상 정책이 가치가 가장 높은 상태를 향한 행동을 유도하기를 원합니다. 이 값은 부트스트랩(bootstrap)을 사용하여 추정할 수 있습니다.

상태(s)는 후속 상태(s')와 s에서 s'으로 가는 것에 대한 보상(R)만큼 가치(V)가 있습니다. 여러 동작(a)과 후속 상태가 있을 수 있으므로 확률(pi)로 가중합됩니다. 
(비결정적(non-deterministic) 환경에서 주어진 행동(action)은 여러 후속 상태를 초래할 수 있습니다. 체스 말을 움직이는 것은 결정적인(deterministic) 게임이기 때문에 이 문제를 고려할 필요는 없습니다.)

후속 상태값은 0과 1사이의 할인 요소(discount factor; gammar)로 할인됩니다.

이것을 수식화하면 다음과 같습니다.

(수식1)

참고:
- 후속 상태 값도 예측(estimate)입니다.
- 다른 예측값을 기반으로 예측을 하기 때문에 상태 평가가 부트스트랩됩니다.
- 이 코드에는 정책 평가 섹션의 뒷부분에서 설명할 동기(synchronous) 매개변수가 있습니다.


## 1.1.2. 구현

In [0]:
print(inspect.getsource(r.evaluate_state))

    def evaluate_state(self, state, gamma=0.9, synchronous=True):
        """
        Calculates the value of a state based on the successor states and the immediate rewards.
        Args:
            state: tuple of 2 integers 0-7 representing the state
            gamma: float, discount factor
            synchronous: Boolean

        Returns: The expected value of the state under the current policy.

        """
        greedy_action_value = np.max(self.agent.policy[state[0], state[1], :])
        greedy_indices = [i for i, a in enumerate(self.agent.policy[state[0], state[1], :]) if
                          a == greedy_action_value]  # List of all greedy actions
        prob = 1 / len(greedy_indices)  # probability of an action occuring
        state_value = 0
        for i in greedy_indices:
            self.env.state = state  # reset state to the one being evaluated
            reward, episode_end = self.env.step(self.agent.action_space[i])
            if synchronous:
           

inspect라는 파이썬 패키지는 함수의 소스를 보고 싶을 때 사용할 수 있습니다.
`inspect(getsource(함수 이름))`를 통해 함수가 어떻게 정의되었는지 볼 수 있습니다. 문서를 이리저리 찾아볼 필요 없이 이 함수를 통해 쉽게 찾아볼 수 있습니다.

여기서는 `r.evaluate_state`함수를 출력하여, RLC.movechess에서 상태평가가 어떻게 구현되어있는지 살펴볼 수 있습니다. 

## 1.1.3. 데모

- 초기 value_function은 각 상태에 0을 할당합니다
- 초기 정책은 각 행동에 대해 동일한 확률을 제공합니다.
- 상태 (0,0)을 평가합니다.

In [0]:
r.agent.value_function.astype(int)

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

상태 (0,0)을 평가합니다.

In [0]:
state = (0,0)
r.agent.value_function[0,0] = r.evaluate_state(state,gamma=1)

In [0]:
r.agent.value_function.astype(int)

array([[-1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

# 1.2. 정책 평가

- 정책 평가는 상태공간에서 각 상태에 대한 doe 상태 평가 행위입니다.
- 모든 상태를 반복하고 value_function을 업데이트합니다.
- 이것은 Sutton과 Barto가 제공하는 알고리즘 입니다.

(그림2)



In [0]:
print(inspect.getsource(r.evaluate_policy))

    def evaluate_policy(self, gamma=0.9, synchronous=True):
        self.agent.value_function_prev = self.agent.value_function.copy()  # For synchronous updates
        for row in range(self.agent.value_function.shape[0]):
            for col in range(self.agent.value_function.shape[1]):
                self.agent.value_function[row, col] = self.evaluate_state((row, col), gamma=gamma,
                                                                          synchronous=synchronous)



inspect.getsource()함수를 이용하여 RLC.movechess에서 정책평가가 어떻게 구현되어있는지 살펴볼 수 있습니다.

In [0]:
r.evaluate_policy(gamma=1)

정책 평가를 실행합니다. 그러면 value_function의 결과는 다음과 같이 종료 상태를 제외한 모든 상태가 -1이 됩니다.

In [0]:
r.agent.value_function.astype(int)

array([[-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  0, -1, -1]])

## 1.2.3. 데모

k_max: 반복 최대 횟수
value_delta_max: 이전 상태의 value_function과 현재 상태의 value_function의 차이가 최대가 되는 값
eps: value_function의 변화량의 최대(value_delta_max)를 측정하여 수렴할지 여부를 판단할 기준값

In [0]:
eps=0.1
k_max = 1000
value_delta_max = 0
gamma = 1
synchronous=True
value_delta_max = 0
for k in range(k_max):
    r.evaluate_policy(gamma=gamma,synchronous=synchronous)
    value_delta = np.max(np.abs(r.agent.value_function_prev - r.agent.value_function))
    value_delta_max = value_delta
    if value_delta_max < eps:
        print('converged at iter',k)
        break

converged at iter 0


In [0]:
r.agent.value_function.astype(int)

array([[-185, -183, -181, -179, -177, -175, -175, -175],
       [-183, -181, -179, -177, -174, -172, -171, -171],
       [-179, -177, -175, -172, -169, -166, -165, -164],
       [-174, -172, -169, -164, -160, -156, -154, -154],
       [-169, -167, -161, -155, -147, -142, -139, -139],
       [-164, -161, -154, -143, -131, -122, -120, -120],
       [-161, -157, -147, -131, -106,  -93,  -92, -102],
       [-160, -156, -145, -126,  -93,    0,  -77,  -93]])

# 1.3. 정책 개선

이제 우리는 상태의 값을 알고있으므로, 행동이 가장 가치가 높은 상태를 향하도록 정책을 개선하고자 합니다. 정책 개선은 단순히 value function과 관련하여 정책을 탐욕스럽게 만드는 행위입니다. 가장 가치있는 상태로 이끄는 동작의 값을 1로하고, 나머지는 0으로 유지하여 이 작업을 수행합니다.


In [0]:
print(inspect.getsource(r.improve_policy))

    def improve_policy(self):
        """
        Finds the greedy policy w.r.t. the current value function
        """

        self.agent.policy_prev = self.agent.policy.copy()
        for row in range(self.agent.action_function.shape[0]):
            for col in range(self.agent.action_function.shape[1]):
                for action in range(self.agent.action_function.shape[2]):
                    self.env.state = (row, col)  # reset state to the one being evaluated
                    reward, episode_end = self.env.step(self.agent.action_space[action])
                    successor_state_value = 0 if episode_end else self.agent.value_function[self.env.state]
                    self.agent.policy[row, col, action] = reward + successor_state_value

                max_policy_value = np.max(self.agent.policy[row, col, :])
                max_indices = [i for i, a in enumerate(self.agent.policy[row, col, :]) if a == max_policy_value]
                for idx in max_indices:
             

In [0]:
r.improve_policy()
r.visualize_policy()

[['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↓', '↙', '↙'],
 ['→', '→', '→', '→', '→', 'F', '←', '←']]


# 1.4. 정책 반복

이제 정책이 안정될 때까지 정책 평가 및 정책 개선을 수행하여 최적의 정책을 찾도록 합니다.

(그림3)


## 1.4.2. 구현

In [0]:
print(inspect.getsource(r.policy_iteration))

    def policy_iteration(self, eps=0.1, gamma=0.9, iteration=1, k=32, synchronous=True):
        """
        Finds the optimal policy
        Args:
            eps: float, exploration rate
            gamma: float, discount factor
            iteration: the iteration number
            k: (int) maximum amount of policy evaluation iterations
            synchronous: (Boolean) whether to use synchronous are asynchronous back-ups 

        Returns:

        """
        policy_stable = True
        print("\n\n______iteration:", iteration, "______")
        print("\n policy:")
        self.visualize_policy()

        print("")
        value_delta_max = 0
        for _ in range(k):
            self.evaluate_policy(gamma=gamma, synchronous=synchronous)
            value_delta = np.max(np.abs(self.agent.value_function_prev - self.agent.value_function))
            value_delta_max = value_delta
            if value_delta_max < eps:
                break
        print("Value function for this po

## 1.4.3. 데모

In [0]:
r.policy_iteration()



______iteration: 1 ______

 policy:
[['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', '↘', '↘', '↘', '↓', '↙', '↙'],
 ['→', '→', '→', '→', '→', 'F', '←', '←']]

Value function for this policy:
[[-5 -5 -5 -5 -5 -5 -5 -5]
 [-5 -5 -5 -5 -5 -5 -5 -5]
 [-4 -4 -4 -4 -4 -4 -4 -4]
 [-4 -3 -3 -3 -3 -3 -3 -3]
 [-4 -3 -3 -3 -3 -3 -3 -3]
 [-4 -3 -3 -2 -2 -2 -2 -2]
 [-4 -3 -3 -2 -1 -1 -1 -2]
 [-4 -3 -3 -2 -1  0 -1 -2]]

 Improving policy:
policy diff: False


______iteration: 2 ______

 policy:
[['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['→', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↗', '→', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↗', '↗', '→', '↘', '↘', '↘', '↓', '↙'],
 ['↗', '↗', '↗', '→', '↘', '↓', '

# 1.5. 비동기 정책 반복



## 1.5.1. 이론

정책평가로 부트스트랩을 사용했습니다. 다른 추정치를 기반으로 추청했습니다. 그래서 우리는 어떤 추정을 하는가요?

이전 정책 평가로부터 부트스트랩을 합니다. 이는 각 상태 값 추정 업데이트가 동일한 정책 평가 반복을 기반으로 한다는 것을 의미합니다. 이를 동기 정책 반복이라고 합니다. 

가장 최근의 추정으로부터 부트스트랩을 합니다. 즉, 추정 업데이트는 이전 또는 현재 value function 또는 이 둘의 조합을 기반으로 할 수 있습니다. 이를 비동기 정책 반복이라고 합니다.

구현은 정책 반복과 동일하며, synchronous=False 인수만 전달해주면 됩니다.

## 1.5.3. 데모

In [0]:
agent = Piece(piece='king')
r = Reinforce(agent,env)

In [0]:
r.policy_iteration(gamma=1,synchronous=False)



______iteration: 1 ______

 policy:
[['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', 'F', '↑', '↑']]

Value function for this policy:
[[-47 -48 -48 -49 -48 -48 -48 -48]
 [-48 -49 -49 -49 -49 -49 -49 -49]
 [-49 -49 -49 -49 -49 -49 -48 -48]
 [-49 -49 -49 -49 -48 -47 -47 -47]
 [-49 -49 -48 -47 -46 -45 -44 -44]
 [-48 -48 -47 -44 -42 -40 -39 -39]
 [-48 -48 -46 -42 -34 -31 -31 -35]
 [-48 -48 -45 -41 -31   0 -27 -33]]

 Improving policy:
policy diff: False


______iteration: 2 ______

 policy:
[['↑', '←', '←', '←', '→', '→', '→', '↑'],
 ['↑', '↖', '↖', '↖', '↗', '↗', '↗', '↑'],
 ['↑', '↖', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↓', '↘', '↘', '↘', '↘', '↘', '↘', '↓'],
 ['↘', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↘', '↘', 

In [0]:
r.agent.value_function.astype(int)

array([[-5, -5, -5, -5, -5, -5, -5, -5],
       [-4, -4, -4, -4, -4, -4, -4, -4],
       [-4, -4, -4, -4, -4, -4, -4, -4],
       [-4, -3, -3, -3, -3, -3, -3, -3],
       [-4, -3, -2, -2, -2, -2, -2, -2],
       [-4, -3, -2, -1, -1, -1, -1, -1],
       [-4, -3, -2, -1, -1, -1, -1, -1],
       [-4, -3, -2, -1, -1,  0, -1, -1]])

# 1.6 가치 반복

## 1.6.1. 이론

가치 반복은 정책 반복에 대한 간단한 매개 변수 수정에 지나지 않습니다. 정책 반복은 정책 평가 및 정책 개선으로 구성됩니다. 정책을 개선하기 전에 수렴될 때까지 정책 평가 단계를 반드시 반복할 필요는 없습니다. 위의 정책 반복이 수렴하는데 428번의 반복이 수행되었습니다(1.2.3. 참고). 대신 1번만 반복하면, 이것을 가치 반복이라고 부릅니다.

## 1.6.3. 데모

이번 데모에서는 루크를 선택해 변화를 살펴봅시다. k_max는 최대 반복횟수를 입력하는 것이었습니다. 여기서는 1번만 반복하기 위해서 1로 설정해줍니다.

In [0]:
agent = Piece(piece='rook') 
r = Reinforce(agent,env)
r.policy_iteration(k=1,gamma=1)  



______iteration: 1 ______

 policy:
[['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↑', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', 'F', '↑', '↑']]

Value function for this policy:
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1  0 -1 -1]]

 Improving policy:
policy diff: False


______iteration: 2 ______

 policy:
[['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '↑', '↑'],
 ['↑', '↑', '↑', '↑', '↑', '↓', '

# 1.7. 다음 이야기

다음에는 Monte Carlo 및 Temporal Difference 기반 방법같이 모델이 없는 방법을 다룰 것입니다. 이러한 방법은 Markov 의사 결정 프로세스의 전환 확률을 모를 때 도움이됩니다.

